<a href="https://colab.research.google.com/github/Hideyuki-Machida/ML_demos/blob/main/%E3%82%BB%E3%83%9E%E3%83%B3%E3%83%86%E3%82%A3%E3%83%83%E3%82%AF%E6%A4%9C%E7%B4%A2_multilingual_e5_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セマンティック検索 - multilingual-e5-large

In [1]:
!nvidia-smi

Sun Jul 23 10:08:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install

In [2]:
!pip install sentence-transformers
!pip install datasets
!pip install fugashi
!pip install ipadic
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=b220f0cbcf60c59b93ee44ad5cbf7dd49cb45bab11e3e9e24f7c8cd2c8e49ed2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta

## データセットをDF化

データセットをPandas DataFrameに変換する。


In [3]:
import os
import urllib.request
import tarfile
import pandas as pd
import glob

# データセットをダウンロード
url = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
filename = "ldcc-20140209.tar.gz"
urllib.request.urlretrieve(url, filename)

# データセットを解凍
with tarfile.open(filename, "r:gz") as tar:
    tar.extractall()

# データセットのディレクトリ
dataset_dir = "text"

# 各記事のファイル名、カテゴリ、本文を読み込み、それらをDataFrameにまとめる
rows = []

for category_dir in os.listdir(dataset_dir):
    category_path = os.path.join(dataset_dir, category_dir)

    if not os.path.isdir(category_path) or category_dir == "LICENSE.txt":
        continue

    for text_file in glob.glob(f"{category_path}/*.txt"):
        with open(text_file, encoding="utf-8") as f:
            lines = f.readlines()
            url = lines[0]
            timestamp = lines[0]
            content = "".join(lines[2:]).strip()
            rows.append({"filename": text_file, "category": category_dir, "content": content, "url": url, "timestamp": timestamp})


df = pd.DataFrame(rows)

# カテゴリ名をIDに変換する辞書を作成
category2id = {category: i for i, category in enumerate(df["category"].unique())}
id2category = {i: category for category, i in category2id.items()}
df["category_id"] = df["category"].map(category2id)

# 結果を表示
df

,filename,category,content,url,timestamp,category_id
0,text/livedoor-homme/livedoor-homme-5511993.txt,livedoor-homme,【特集/JOURNEY】 刺激的で優しいアラブの国へ （1/8）\nみなさんはドバイにどのよ...,http://news.livedoor.com/article/detail/551199...,http://news.livedoor.com/article/detail/551199...,0
1,text/livedoor-homme/livedoor-homme-6429261.txt,livedoor-homme,世界的に盛り上がる、“旧車カスタム”というカルチャー\nカーカスタムの世界で、いま最も注目が...,http://news.livedoor.com/article/detail/642926...,http://news.livedoor.com/article/detail/642926...,0
2,text/livedoor-homme/livedoor-homme-5825810.txt,livedoor-homme,年収1000万円のビジネスパーソンに聞いた「子供を進学させたい大学ランキング ベスト20」 ...,http://news.livedoor.com/article/detail/582581...,http://news.livedoor.com/article/detail/582581...,0
3,text/livedoor-homme/livedoor-homme-4583815.txt,livedoor-homme,フォーエバー２１来店者数が４００万人突破\n ファストファッションチェーン「フォーエバー２１...,http://news.livedoor.com/article/detail/458381...,http://news.livedoor.com/article/detail/458381...,0
4,text/livedoor-homme/livedoor-homme-5825811.txt,livedoor-homme,年収1000万円のビジネスパーソンに聞いた「グローバル化社会への準備と心構え（前編）」 - ...,http://news.livedoor.com/article/detail/582581...,http://news.livedoor.com/article/detail/582581...,0
...,...,...,...,...,...,...
7371,text/kaden-channel/kaden-channel-6207425.txt,kaden-channel,五輪イヤーの今年、日立「Wooo」の新製品はさらに高画質化、省エネ性アップ【売れ筋チェック】...,http://news.livedoor.com/article/detail/620742...,http://news.livedoor.com/article/detail/620742...,8
7372,text/kaden-channel/kaden-channel-6161620.txt,kaden-channel,さんまには教えなかった「老後」について、ついにメッシが語った！【話題】\n「TOYOTA プ...,http://news.livedoor.com/article/detail/616162...,http://news.livedoor.com/article/detail/616162...,8
7373,text/kaden-channel/kaden-channel-6108618.txt,kaden-channel,テレビ離れなんて関係ない！「家政婦のミタ」人気がすごすぎる！！【話題】\nテレビ離れや視聴率...,http://news.livedoor.com/article/detail/610861...,http://news.livedoor.com/article/detail/610861...,8
7374,text/kaden-channel/kaden-channel-6853386.txt,kaden-channel,曲がり角の先が見えるカメラ【サイエンスニュース】\nフィクションの中だけだと思われていた技術...,http://news.livedoor.com/article/detail/685338...,http://news.livedoor.com/article/detail/685338...,8


## ベクトルDBの作成

Qdrant、Chromadbを使用して、ベクトルDBを作成する。

In [4]:
from langchain.docstore.document import Document

docs = []

for index, row in df.iterrows():
    doc = Document(page_content="query: " + row.content, metadata=row.to_dict())
    docs.append(doc)

len(docs)

7376

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

# モデルの準備
MODEL_NAME = "intfloat/multilingual-e5-large"

embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)

In [6]:
# DB作成

#"""
!pip install qdrant-client

from langchain.vectorstores import Qdrant

db_client = Qdrant.from_documents(
    docs, embeddings,
    path="./local_qdrant_db",
    collection_name="livedoor_documents",
)

"""
!pip install chromadb

from langchain.vectorstores import Chroma

db_client = Chroma.from_documents(
    docs, embeddings,
    path="./local_chroma_db",
    collection_name="livedoor_documents",
)
"""

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.

'\n!pip install chromadb\n\nfrom langchain.vectorstores import Chroma\n\ndb_client = Chroma.from_documents(\n    docs, embeddings,\n    path="./local_chroma_db",\n    collection_name="livedoor_documents",\n)\n'

## セマンティック検索


In [7]:
query = "ダルビッシュについて"
found_docs = db_client.similarity_search_with_score(query, k=5)

for i, doc in enumerate(found_docs):
    print(doc[0])
    print("category_id: ", doc[0].metadata["category_id"])
    print("score: ", doc[1])
    print("----")

page_content='query: 試合中、ピンチの場面でダルビッシュが見せた“ある変化”とは？\n日本時間8日早朝、テキサス・レンジャーズのダルビッシュ有がオープン戦に初登板を果たし、サンディエゴ・パドレスを相手に2回無失点、3三振でメジャー・デビューを飾った。\n\n同夜、テレビ朝日「報道ステーション」では、野球解説者・工藤公康氏がダルビッシュのピッチングを振り返りつつ、試合中に彼が見せた“ある変化”を指摘した。\n\n「私が注目したのは、ダルビッシュ投手が自分の投げたい球種を投げることができたのかというところです。チームに、キャッチャーに、自分の特徴を理解して貰う。いわば、マウンドの上で自己紹介ができたかというところ」と戦前にポイントを語っていた工藤氏は、試合後「全球種を投げることができました。やりたいことは全てできたという風に思います」と評価した。\n\nすると、番組内では、ダルビッシュが1回2アウト2塁のピンチになった場面で、工藤氏がリアルタイムで解説する音声が紹介された。\n\n「セットの入り方が変わったの、分かった？」と切り出した工藤氏は、ピンチを迎えたダルビッシュの構えが変わったことを指摘。「膝を右に少し曲げてるでしょ？ ギアがセカンドにいったのかな」と語り、「ここで打たれたくないから抑えるのにちょっと力を入れるかって、ちゃんと（膝を）はめた状態でいけば力が出るっていうのを確かめたのかもしれない」と説明した。\n\n膝の角度を変えて、より力を込めて投球したというダルビッシュ。工藤氏は“ギアをセカンドに……”と（本人は知ってか知らずか）人気アニメ「ワンピース」のルフィの如く表現したが、具体的には「実はあのピンチの場面、ダルビッシュ投手がどうして膝を曲げてギアを上がった状態で強いボールを投げることができたのか。まず重たいものがあるとします。膝を伸ばした状態で押そうとした時に、どうしても下半身の力が使えず、上体だけで押してしまうので重たいものは押せませんよね。膝を曲げて重心を落とした状態で押そうとすると、下の力がしっかり重たいものに伝わって強くなる。そういった意味でダルビッシュ投手はあの時にそれができたと思うんですけど、実はダルビッシュ投手、日本で投げている時、ピンチの時もそういう投球ができたんです」と詳細を語った。\n\n・松井秀喜はどんな決断をす

In [8]:
query = "ダルビッシュの結婚について"
found_docs = db_client.similarity_search_with_score(query, k=5)

for i, doc in enumerate(found_docs):
    print(doc[0])
    print("category_id: ", doc[0].metadata["category_id"])
    print("score: ", doc[1])
    print("----")

page_content='query: ダルの紗栄子擁護に「じゃあなぜ離婚したんだ」の声\n米レンジャーズ入団会見から帰国したダルビッシュ有投手が、23日、自身のブログで紗栄子との離婚の真意について触れたことがネットで話題になっている。\n\n同ブログでは、離婚について「子供たちの事を第一に考えて、二人でゆっくり話し考えた結果」と伝え、紗栄子については「彼女に関してメディア等で根も葉もない話が出てましたが事実がほぼゼロ」「彼女は献身的に自分を支えようと頑張ってくれてましたし、家庭内の仕事も一生懸命こなしてくれてました」と記載している。メディアで報じられていた慰謝料や養育費に関しても「法外な慰謝料、養育費を求められたこともありません」と言い切った。\n\nさらに「自分と彼女との関係も良好なので子供たちには沢山会えると思います」「彼女の母のように、さらに愛に溢れた母親へなることでしょうね」とネット上で批判の声が後を絶たない紗栄子を擁護し“円満離婚”をアピールしているようだった。\n\nこのブログ記事を受け、ネット掲示板では「円満なら離婚すんな」「サエコ擁護しているようで嫌な含みを持たせている」「本音を聞きたいな」など、様々な反応が書き込まれており、円満離婚をアピールするダルビッシュの真意を図りかねているユーザーが多いようだ\n\n「応援していきたい」とお互いにエールを送っているこの2人だが、今後もその動向から目が離せない。\n\n■紗栄子の写真ギャラリー \n\n【関連記事】\n・ダルが法外な養育費報道に反論、“紗栄子が要求”は「根も葉もない話」。\n・ダルビッシュ、ブログで紗栄子を擁護「自分も彼女を応援したい」\n【関連情報】\n・カナ速\n・ダルビッシュ有オフィシャルブログ' metadata={'filename': 'text/topic-news/topic-news-6217249.txt', 'category': 'topic-news', 'content': 'ダルの紗栄子擁護に「じゃあなぜ離婚したんだ」の声\n米レンジャーズ入団会見から帰国したダルビッシュ有投手が、23日、自身のブログで紗栄子との離婚の真意について触れたことがネットで話題になっている。\n\n同ブログでは、離婚について「子供たちの事を第一に考えて、二人でゆっくり話し考えた結果」と

In [9]:
query = "ダルビッシュと紗栄子について"
found_docs = db_client.similarity_search_with_score(query, k=5)

for i, doc in enumerate(found_docs):
    print(doc[0])
    print("category_id: ", doc[0].metadata["category_id"])
    print("score: ", doc[1])
    print("----")

page_content='query: 紗栄子がダルビッシュ有投手との2ショット写真公開、「あざとい」と非難\n27日、タレントの紗栄子が自身のブログで元夫であるダルビッシュ有投手との2ショット写真を公開し、ネット掲示板で話題になっている。\n\n同記事では、紗栄子が自身のブログで「BEST FRIEND。」と題した記事に「今でも支えてくれる有くんに感謝」と綴り、現在のダルビッシュとの関係を語っている。さらに紗栄子が笑顔でケーキを手に、ダルビッシュとの2ショット画像を掲載し、つい1月19日に離婚を発表したとは思えないような良好な関係をアピールしている。\n\nこの記事を受け、ネット掲示板では「なるほど、"今でも（お金で）支えてくれる有くんに感謝"てことか」「ダルをトコトン踏み台にして恐ろしい女だな…もう開放してやれよ…」「こうやってネタ投下してブログのアクセス増やして、あざといイメージ作りに余念が無いな」など、ブログ内容に他意を感じたユーザーからの紗栄子批判が続出した。\n\nブログに掲載されていた画像についても、「ダルの目が明らかに死んでるじゃねーかwww」「サエコの意味不明な笑顔にダルの感情を噛み殺したような迷惑そうな表情…」「ダル…泣いてもいいんだぜ…」など、ダルビッシュへの同情が相次いだ。\n\nまた、「仲良すぎだろww離婚すんなよ」など、ブログで綴られていたような良好な関係が本当に保てているなら、なぜ離婚したのだろうと、疑問を呈する声も上がっている。\n\n■ダルビッシュ有の写真ギャラリー\n■紗栄子の写真ギャラリー\n\n【関連記事】\n・紗栄子とダルビッシュの2ショット写真公開！「今でも支えてくれる有くんに感謝」 (シネマトゥデイ)\n・渡米直前のダルビッシュ有\u3000子供たちとの面会時間は90分 (livedoorニュース)\n・ビキニ姿「紗栄子」が好アクセス 「嫌われ者」でも高い注目度(livedoorニュース)\n\n【関連情報】\n・紗栄子（Saeko） オフィシャルブログ \u3000- 公式ブログ' metadata={'filename': 'text/topic-news/topic-news-6320098.txt', 'category': 'topic-news', 'content': '紗栄子がダルビッシュ有投手との2シ

In [10]:
query = "結婚について"
found_docs = db_client.similarity_search_with_score(query, k=5)

for i, doc in enumerate(found_docs):
    print(doc[0])
    print("category_id: ", doc[0].metadata["category_id"])
    print("score: ", doc[1])
    print("----")

page_content='query: 結婚を国民の義務にする？\n内閣府が公表した2012年版「子ども・子育て白書」によると、2010年時点で50歳までに一度も結婚したことのない「生涯未婚率」は、男性は20.14％、女性は10.61％と、いずれも過去最高となった。\n結婚をしない理由について、25歳から34歳の男性の3割が、「結婚資金が足りない」ことを挙げており、20代の独身女性は相手の年収の少ないことや子育ての大変さを挙げている。\n\nやっぱり問題はお金なのか？\u3000\n\n「うちは男兄弟がいないので、親の老後の面倒は私が見たいんです。転勤のない職種で実家の近くに住んでくれる人を探しているのですが、なかなかいなくて」\nと嘆くアキさん（28歳）は、結婚相手に高収入を求めていない。\n\nしかし転勤がなくて、嫁の実家の近くに住んでくれる人となると、探すのはなかなか難しそうだ。ところが親戚の紹介でやっと条件通りの男性が現れた。相手は公務員で次男。でもアキさんは断ったそうだ。\n\n「草食系で会話もつまらなかったし、見合いに母親がついてきたのもマザコンみたいで、2回目に会うのはお断りました」\nちなみに見合いにはアキさんの母親も同行している。男性だけがマザコンというのは、ちょっとおかしい気がするのだが。\n\n婚活は思い立ったときが適齢期。「再婚でも子どものいる人でもいいから誰かいないかしら」と、結婚相手を探しているのは大手商社に勤務する久美子さん（63歳）。\n若いころは美人でモテモテだった久美子さん。彼はいつもいたし、プロポーズもされたけど、仕事が楽しく家庭に入る気にはならなかった。今も60代には見えない若さと美しさだが、そろそろ定年を迎える今、老後、一緒に旅行をする男性がほしいという。\nたまたま筆者の知人男性に60歳の独身男性がいたので紹介しようと写真を見せたら、容姿だけは譲れないと断られた。\n\nつくづく思ったのは、婚活がうまくいかないのは、選びすぎだということ。\n条件を全て満たす相手などいないのだから、どこかで妥協しないと結婚はできない。\n受験も就活も希望が叶った人はわずかだ。自分にぴったりと思える仕事についた人もごくわずかだ。それでもみんな、毎朝、頑張って起きて出社し、仕事をしているではないか。\n\n久美子さんと同世代の既婚女性の和子さ

In [11]:
query = "iPhoneアプリについて"
found_docs = db_client.similarity_search_with_score(query, k=5)

for i, doc in enumerate(found_docs):
    print(doc[0])
    print("category_id: ", doc[0].metadata["category_id"])
    print("score: ", doc[1])
    print("----")

page_content='query: ようやくiPhoneでもChromeが使えるように！【iPhoneでチャンスを掴め】\nGoogleと言えばiPhone対抗とも言えるAndroidを出してる総本山です。ライバルということもあり、便利なWebブラウザであるGoogle ChromeのiOS版はリリースされていませんでした。かなり便利なWebブラウザであるChromeをiPhoneで使いたいという人も多かったのではないだろうか。そんな方に朗報！ようやくiPhone向けのChromeが登場しました。こんな人におすすめ：\nパソコンでChromeを使っている人\nこんなときにおすすめ：\nタブをたくさん開いて検索したいとき\n\n\n\nアプリ名：Chrome\nカテゴリ：ユーティリティ\n価格：無料\n\n\nパソコンではChrome使われている方も多いと思います。ChromeのiPhoneアプリ版が登場しました。これは便利になる予感！\n\nパソコンでChromeを使っていない人はこのアプリをダウンロードしたら、パソコンでもChromeを使いたくなりそう。\n\n無限にタブを開く事ができます。いろいろ調べ物のときには助かりますよね。\n\n\n同じIDでログインすれば、パソコンとの同期もできちゃいます。\n電車の中で見たサイトを家に帰ってからじっくり見ることも可能です。\n\nシークレットモードでアクセスもできたり。こちらは、履歴やCookieに残したくない検索を調べるときに使えそうです。サクサク動くので、iPhoneがますます便利になりましたね。\n\n\n■RainbowApps - iPhoneアプリ開発を学ぶ\nなら1000人が受講したEagleが運営する東京校が一番！\n■成長率 888% 毎日 50万台増える Androidアプリに参入しよう。Androidマスターコース\n■人気のAngryBirds風アプリやスーパーマリオ風アプリが簡単に作れちゃう。ゲームアプリコース\n■iPhoneの日米人気アプリランキングをチェックするからRainbowApps\n\n■RainbowAppsの記事をもっと読む' metadata={'filename': 'text/it-life-hack/it-life-hack-6749297.txt', 'cat

In [12]:
query = "ハリウッドについて"
found_docs = db_client.similarity_search_with_score(query, k=5)

for i, doc in enumerate(found_docs):
    print(doc[0])
    print("category_id: ", doc[0].metadata["category_id"])
    print("score: ", doc[1])
    print("----")

page_content='query: ブラッド・ピットのジンクスは家族にあり！ \n11月10日、映画『マネーボール』の来日記者会見が、グランドハイアット東京にて行われ、主演と製作を務めるブラッド・ピットが「非常に誇りに思っている作品」と述べるなど同作の魅力を語った。ハリウッドの超大物スターが久しぶりに来日し、前夜のジャパンプレミアも盛り上がったが、この会見にもスチールカメラマン100名、記者300名、ムービー20台が集結。この日、花束贈呈ゲストには女優の吹石一恵が登場。彼女の父親が元プロ野球選手ということで「（お父さんが作品を見たら）感想を教えてほしい」とブラッドは語りかけた。\n\n今回は家族8人揃って来日したブラッド。「家族を持ったことは僕に変化をもたらしました。日々が戦いであり、挑戦の場。家族の元に戻ると、ホッと一息付けるので、とても貴重な存在です」と、穏やかな表情で語った。また、野球にはつきものである“ジンクス”に話題が及ぶと、「ひとりで飛行機に乗る場合は、家族の物を何か持って乗るようにしています。それが僕のジンクスです」と明かした。\n\n実話を基に映像化した本作は、野球が題材になっているものの、試写を見た観客からは“野球に興味がないけれどとても面白かった”という声が多く。またハリウッドでも作品にならって「ホームラン級」の出来栄えと評価されるなど、アカデミー賞作品賞の呼び声が高い。ブラッドは「僕もあまり野球を知らないで、この作品をつくりました」と前置きし、「野球は映画の背景。これは正義の話であり、負け犬というレッテルをはることや、失敗や成功といった価値観は正しいのかどうか、そういうことを問う映画」だと述べた。そして現代の若者たちに対し「常識に疑問を持つことが大事。問題がある場合は、その解決策を見出すことが一番大切だと思います」とメッセージを送った。\n\nなおかつ「（ブラッドが演じた主人公の）ビリーは、同じ土俵で公平に戦って、勝利したいのです。財源力に差があるチームが戦うことは不公平だと。そこからビリーは、いかに勝つかを考え始めます。約150年間続いた球界のルールや旧来のスカウト方法が今通用するのか？\u3000成功と敗北は何で決まるのか？ を突き詰めていきます。そして数字によって、今まで価値がないとされていた人材に価値を見出して発掘していくのです」

In [13]:
query = "ハリウッドに関連したニュース"
found_docs = db_client.similarity_search_with_score(query, k=5)

for i, doc in enumerate(found_docs):
    print(doc[0])
    print("category_id: ", doc[0].metadata["category_id"])
    print("score: ", doc[1])
    print("----")

page_content='query: 【速報】「第84回アカデミー賞」、注目の“映画愛”対決は『アーティスト』が制す\n\u30001982年の設立より毎年、アメリカで映画製作に携わる俳優や監督など約6,000人から構成される「映画芸術科学アカデミー（AMPAS）」の会員によって、前年に公開された作品を対象に各賞を選定する「アカデミー賞」。84回目を迎える今年は、日本時間27日10時半よりロサンゼルスにて授賞式が開催された。\n\n\u30001月24日に発表されたノミネートでは、先日5年振りの来日を果たした巨匠マーティン・スコセッシ監督による『ヒューゴの不思議な発明』（3月1日公開）が最多11部門、次いで前哨戦の「第69回ゴールデン・グローブ賞」で最多3冠を獲得した『アーティスト』（4月1日公開）が10部門でノミネートを果たし、“映画愛”対決が注目の的に。いずれも最多5部門を受賞したが、作品賞および監督賞は『アーティスト』が手にした。\n\n\u3000また、主演男優賞では、ジョージ・クルーニー（『ファミリー・ツリー』）とブラッド・ピット（『マネーボール』）の2大俳優の対決に注目が集まっていたが、作品賞と同じく『アーティスト』のジャン・デュジャルダン（アーティスト）が獲得。一方の主演女優賞では、同賞に史上最多となる17回のノミネートを誇るメリル・ストリープ（マーガレット・サッチャー\u3000鉄の女の涙）が3度目のオスカー受賞を果たした。各部門の受賞結果は、以下の通り（受賞作は★印）。\n\n・「第84回アカデミー賞」特集ページ\n\n作品賞\n★アーティスト（4月1日公開）\n・ファミリー・ツリー（5月18日公開）\n・ものすごくうるさくて、ありえないほど近い（公開中）\n・ヘルプ\u3000〜心がつなぐストーリー〜（3月31日公開）\n・ヒューゴの不思議な発明（3月1日公開）\n・ミッドナイト・イン・パリ（5月26日公開）\n・マネーボール（公開中）\n・ツリー・オブ・ライフ（3月7日BD&DVD発売）\n・戦火の馬（3月2日公開）\n\n主演男優賞\n・デミアン・ビチル（明日を継ぐために）\n・ジョージ・クルーニー（ファミリー・ツリー）\n★ジャン・デュジャルダン（アーティスト）\n・ゲイリー・オールドマン（裏切りのサーカス）\n・ブラッド・ピット（マネ

In [14]:
query = "ハリウッドのニュース"
found_docs = db_client.similarity_search_with_score(query, k=5)

for i, doc in enumerate(found_docs):
    print(doc[0])
    print("category_id: ", doc[0].metadata["category_id"])
    print("score: ", doc[1])
    print("----")

page_content='query: 【速報】第69回ゴールデン・グローブ賞、『アーティスト』が最多3冠\n\u30001944年より映画のアワードとして開始し、1955年からはテレビの部を新設。世界で活躍する約90名の会員が所属するハリウッド外国人記者協会によって、前年に発表された最新の映画・テレビ番組を賞賛する「ゴールデン・グローブ賞」。現地時間1月15日には「第69回ゴールデン・グローブ賞」授賞式が開催され、最多6部門にノミネートされた『アーティスト』が3冠を達成した（受賞作は★印）。\n\nセシル・B・デミル賞\n★モーガン・フリーマン\n\nドラマ部門\u3000作品賞\n★ファミリー・ツリー\n・ヘルプ\u3000〜心がつなぐストーリー〜\n・ヒューゴの不思議な発明\n・スーパー・チューズデー 〜正義を売った日〜\n・マネーボール\n・戦火の馬\n\nドラマ部門\u3000女優賞\n・グレン・クローズ「ALBERT NOBBS」\n・ヴィオラ・デイヴィス「ヘルプ\u3000〜心がつなぐストーリー〜」\n・ルーニー・マーラ「ドラゴン・タトゥーの女」\n★メリル・ストリープ「マーガレット・サッチャー\u3000鉄の女の涙」\n・ティルダ・スウィントン「少年は残酷な弓を射る」\n\nドラマ部門\u3000男優賞\n★ジョージ・クルーニー「ファミリー・ツリー」\n・レオナルド・ディカプリオ「Ｊ・エドガー」\n・マイケル・ファスベンダー「SHAME - シェイム -」\n・ライアン・ゴズリング「スーパー・チューズデー 〜正義を売った日〜」\n・ブラッド・ピット「マネーボール」\n\nミュージカル・コメディ部門\u3000作品賞\n・50/50 フィフティ・フィフティ\n★アーティスト\n・BRIDESMAIDS\n・ミッドナイト・イン・パリ\n・マリリン\u30007日間の恋\n\nミュージカル・コメディ部門\u3000女優賞\n・ジョディ・フォスター「おとなのけんか」\n・シャーリーズ・セロン「ヤング≒アダルト」\n・クリステン・ウィグ「BRIDESMAIDS」\n★ミシェル・ウィリアムズ「マリリン\u30007日間の恋」\n・ケイト・ウィンスレット「おとなのけんか」\n\nミュージカル・コメディ部門\u3000男優賞\n★ジャン・デュジャルダン「アーティス

In [15]:
query = "ムカデ人間について"
found_docs = db_client.similarity_search_with_score(query, k=5)

for i, doc in enumerate(found_docs):
    print(doc[0])
    print("category_id: ", doc[0].metadata["category_id"])
    print("score: ", doc[1])
    print("----")

page_content='query: 【編集部的映画批評】みんなで繋がれば怖くない、自己責任爽やかホラー\n\u3000今回カルト映画のフジモトが紹介するのは、『ムカデ人間』。想像するのも恐ろしい人体実験を実行に移してしまうマッドサイエンティストと、実験の結果誕生した“ムカデ人間”のサバイバルを描くカルトホラーだ。続編『ムカデ人間2』公開も決まったので、次回作への免疫をつけるべく、爽やかに1作目を紹介！\n\n『ムカデ人間』\n\u3000舞台はドイツ郊外。ニューヨークからの旅行者リンジーとジェニーはレンタカーで道に迷い、タイヤがパンクして森の中で孤立してしまう。携帯電話もつながらず、二人は歩いて助けを求めることに。降りしきる雨に凍えながら発見した人里はなれた邸宅には、かつてシャム双生児の分離で名を馳せた、元外科医のヨゼフ・ハイター博士(ディーター・ラーザー)が住んでいた。博士のおかしな様子に怯えた二人だったが、時すでに遅く睡眠薬で眠らされてしまう。気がつくと、2人は日本のやくざ・カツローとともにベットに縛られた動けない状態。3人の目の前で、ハイター博士は世にも恐ろしい演説を始める。今からリンジー、ジェニー、カツローの3人の口と肛門をつなぎ合わせ、手足の腱を切り、人類史上初の“ムカデ人間”を創り出す、と。やがて、世にも恐ろしい、常軌を逸した手術が始まるのだった…。（作品情報へ）\n\n\n爽やか！グロくない！自己責任制ホラー\n\u3000さて、誰もが気になるのが、実際「口と肛門を繋ぐとどうなるのか」ということだ。このあまりにおぞましい設定に、観ること自体を敬遠する方も多いだろう。劇中のハイター博士の説明を引用するとこうだ。\n「A体、B体、C体の3つの人体の、ヒザの皿のじん帯を切除して四つん這いにさせ、A体の肛門とすべての歯を抜いたB体の口唇、同じくB体の肛門とC体の口唇を縫合して一体化させ、養分はAが口から摂取し、Bを通過してCの肛門から排泄させる」\u3000\n\u3000実に恐ろしい発想である。しかし、設定だけに惑わされないでほしい。実はこの作品、びっくりするほど爽やかな映画なのだ。劇中では手術する様子は出てくるものの、いわゆるスプラッター映画のような直接的な残酷表現はほぼ無いと言っていい。最低限の部分を見せ、想像させる表現に徹しているのである。例えば